In [137]:
from wordcloud import WordCloud
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib 
from scipy import optimize
from scipy import stats
plt.style.use("ggplot")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
#plt.rcParams['font.family'] = 'IPAexGothic'

import warnings
warnings.filterwarnings('ignore') # warningが出ないように設定
pd.set_option("display.max_rows", None) # pandasの表示上限をなくす
pd.set_option("display.max_columns", None) # pandasの表示上限をなくす
import pickle
import gspread
import json
from oauth2client.service_account import ServiceAccountCredentials
import collections
import gensim

model = gensim.models.Word2Vec.load("../latest-ja-word2vec-gensim-model/word2vec.gensim.model")

In [138]:
file = "../relation_vec_fold/10.binaryfile"
f = open(file,'rb')
mean_vec_list = pickle.load(f)

In [139]:
# (1) Google Spread Sheetsにアクセス
def connect_gspread(jsonf,key):
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
    gc = gspread.authorize(credentials)
    SPREADSHEET_KEY = key
    worksheet = gc.open_by_key(SPREADSHEET_KEY).sheet1
    return worksheet

In [140]:
def result_vec_list_generator(imput_word_list):
    result_vec_list  = []

    for w in imput_word_list:

        try:
            w_to_v = model.wv[w]

            for mean_vec in mean_vec_list:
                result_vec_plus = w_to_v - mean_vec 
                result_vec_minus =   w_to_v + mean_vec 
    
                result_vec_list.append(result_vec_plus)
                result_vec_list.append(result_vec_minus)
        except:
            print("Error", w)

    return result_vec_list

In [141]:
# 関係ベクトルと類似度が高い単語をn個上位表示
def output_word_list_generator(result_vec_list):
    output_word_list = []
    max_n = 2

    for vec in result_vec_list:
        most_similar = np.array(list(model.wv.most_similar([vec], [], max_n)))
    
        output_word_list.extend(most_similar[:,0].tolist())
    return output_word_list

In [147]:
def ggg(list_taple):
    l = []
    for taple in list_taple:
        a = taple[0]
        l.append(a)
    return l

In [143]:
# ここでjsonfile名と2-2で用意したkeyを入力
jsonf = "quickstart-1595337771922-22c2ceb47d3c.json"
spread_sheet_key = "1WKpQ4sxL_IP9Fm6551xDRfhxhDLzmgGAG3FCzDFmamI"
worksheet = connect_gspread(jsonf,spread_sheet_key)

In [154]:
for i in range(2, 33):
    answers_list = worksheet.row_values(i)
    imput_word_list = answers_list[2:]
    answer_word = answers_list[1]
    print("answer", answer_word)
    print("imput", imput_word_list)
    
    result_vec_list = result_vec_list_generator(imput_word_list)
    output_word_list = output_word_list_generator(result_vec_list)
    
    for imput_word in imput_word_list:
        output_word_list = [output_word for output_word in output_word_list if output_word != imput_word]
    
    c = collections.Counter(output_word_list)
    dic = dict(c)
    dic2 = sorted(dic.items(), key=lambda x:x[1], reverse=True)
    l = ggg(dic2[:30])
    print(l)
    
    if answer_word in l: 
        print("成功**************************")
    
    print("　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　")

answer LIXIL
imput ['トステム', 'INAX', '新日軽', 'サンウエーブ工業', '東洋エクステリア', 'YKK', '大建工業']
Error サンウエーブ工業
Error 東洋エクステリア
Error 大建工業
['光洋', 'モンテヴェルディ', '原典版', 'ハート・ファウンデーション', '楽市', '親和銀行', '野平一郎', 'ことぶき', 'フジテレビジョン', '交響曲', '東三河', 'カンパニー', 'みつい', '当社', 'イアン', '錦海', 'ケーブルネットワーク', '商事', 'アミュプラザ', '楽譜', '光丘', 'カラヤン', '天竜川', '惣七', '三和銀行', 'ダイムラーベンツ', '宣伝会議', 'オーケー', '鉄男', 'JCN']
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
answer ハラール
imput ['イスラム教', 'イスラム', 'アラビア', 'アジア', 'インドネシア']
['畿内', 'ヨーロッパ', 'アメリカ', 'ナイル川', 'スカンジナビア', 'メソポタミア', 'マレーシア', 'カナダ', '四国', 'イスラーム', 'オセアニア', '沿岸地域', 'ミャンマー', 'しんとう', '東国', 'ブリテン', '南米', '東アジア', 'ノルウェー', 'ユダヤ教', 'かれ', 'ムスリム', '諸王', '中東', 'ポー', '西アジア', 'ウガリット', 'ナボコフ', '東南アジア', '西日本']
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
answer ダイドードリンコ
imput ['キリンビバレッジ', 'ダイドーブレンド', 'サントリー', '日本たばこ産業', 'JT', 'コカ・コーラ']
Error ダイドーブレンド
['当社', '資生堂', '森永', '松下', '北陸', '原典版', '本舗', 'デッカ', '日清食品', 'JA', 'P&G', 'テーブルマーク', '錦海', '東京ガス', 'コカコーラ', 'ペプシ', 'サ

answer ホテルオークラ
imput ['東京', 'ジ・オークラ・トーキョー', '帝国ホテル', 'パークハイアット東京', '椿山荘']
Error ジ・オークラ・トーキョー
Error パークハイアット東京
['大阪', '演奏会', '神田', '京都', 'エルガー', 'ニューヨーク', 'チャイコフスキー', '旭川', 'JOAK', '世界初演', '中央卸売市場', 'ショウ', 'NHK放送センター', 'あべの', 'NHK札幌放送局', 'ロンドン', '神戸', '東京放送局', 'ハリス', '伊藤', '大阪放送局', '日比谷公園', '蔵屋敷', '会所', 'ニューオータニ', 'パーキンス', 'LOVE FM', 'ウォーターフロントパーク', 'キング・オブ・ザ・リング', '正雄']
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
answer エイチ・アイ・エス
imput ['長崎', '佐世保', 'ハウステンボス']
['演奏会', '横須賀', 'チャイコフスキー', '伊勢', 'さくら', '淀川', '舞鶴', '関門海峡', '東京湾', 'ベートーヴェン', '名古屋港', '熱海', '石巻', '柏崎', 'ひろし', '明石', '交響曲第9番', '房総', '銚子', '下田', '舟運', '朝鮮通信使', '短波放送', 'STV', '大宰府', '礼子', '鷹島', '草津', '斜里', '正夫']
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　
answer PISA
imput ['OECD', 'Programme', '日本']
['経済協力開発機構', '米国', '批評家', '欧州', 'Broadcast', '統計', 'EMEA', '衛星放送', '国際水路機関', 'アメリカ', '寛司', 'CIS', '環境基準', '畿内', '家計調査', 'Program', '英国', '韓国', 'ふみ', '東北地方', '台湾出身', 'HDI', '平成17年度', 'フィッシャー', '人間開発指数',

In [35]:
# 複数の単語で類似度の高いものを上から１０個表示する
most_similar = np.array(list(model.wv.most_similar(imput_word_list, [], 10)))[:,0]
most_similar.tolist()

['メチルアミン',
 'ボラン',
 'グルコサミン',
 'リン酸エステル',
 'グリコサミノグリカン',
 '芳香族炭化水素',
 'ポリアミド',
 'トリアジン',
 'フェノール類',
 'アダマンタン']

In [128]:
a = (1, 23)

In [130]:
a[0]

1